In [1]:
import pandas as pd
import numpy as np
from datetime  import datetime  
from datetime import timedelta  
import lightgbm as lgb
from sklearn import preprocessing, metrics
from sklearn.model_selection import ParameterGrid
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
import mlflow.lightgbm

In [2]:
X = pd.read_pickle('../01_preprocessed_data/X.pkl')

In [3]:
X[['d', 'days_from_start']]

,d,days_from_start
700176,d_56,55
700177,d_56,55
700178,d_56,55
700179,d_56,55
700180,d_56,55
...,...,...
853715,d_1941,1940
853716,d_1941,1940
853717,d_1941,1940
853718,d_1941,1940


In [4]:
lags = [1, 2, 3, 4, 5, 6, 7]
lag_cols = [f"lag_{lag}" for lag in lags ]
for lag, lag_col in zip(lags, lag_cols):
    X[lag_col] = X.groupby("id")["demand"].shift(lag)


In [5]:
lags = [1, 2, 3, 4, 5, 6, 7, 28]
windows = [7, 28, 30, 31]
for window in windows:
    for lag,lag_col in zip(lags, lag_cols):
        X[f"rmean_{lag}_{window}"] = X[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(window).mean())

In [6]:
X.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'demand', 'date', 'weekday', 'month', 'event_name_1', 'event_type_1',
       'year', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI',
       'start_date', 'days_from_start', 'start_date_from_start', 'sell_price',
       'ADI', 'CV2', 'demand_type', 'avg_weekday_demand', 'avg_demand',
       'max_demand', 'quantile025_week', 'max_weekday_demand', 'avg_price',
       'cheaper_than_usual', 'lag_28', 'lag_29', 'lag_35', 'rmean_28_7',
       'rmean_28_28', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6',
       'lag_7', 'rmean_1_7', 'rmean_2_7', 'rmean_3_7', 'rmean_4_7',
       'rmean_5_7', 'rmean_6_7', 'rmean_7_7', 'rmean_1_28', 'rmean_2_28',
       'rmean_3_28', 'rmean_4_28', 'rmean_5_28', 'rmean_6_28', 'rmean_7_28',
       'rmean_1_30', 'rmean_2_30', 'rmean_3_30', 'rmean_4_30', 'rmean_5_30',
       'rmean_6_30', 'rmean_7_30', 'rmean_1_31', 'rmean_2_31', 'rmean_3_31',
       'rmean_4_31',

In [7]:
first_day = pd.Timestamp(X['date'].values.min())
last_day = pd.Timestamp(X['date'].values.max())



In [8]:
last_X_day = pd.Timestamp(X.date.values.max())

In [9]:
last_X_day

Timestamp('2016-05-22 00:00:00')

In [10]:
first_train_day = first_day + timedelta(days = 365)
n_days = last_day-first_train_day
n_days = n_days.days
n_val_days = 28# round(n_days*val_size)
n_test_days = 28
first_val_day = first_train_day + timedelta(days = n_days - n_val_days - n_test_days + 1)
first_test_day = first_val_day +  timedelta(days = n_test_days)

In [11]:
first_val_day - first_train_day

Timedelta('1465 days 00:00:00')

In [12]:
first_test_day - last_day

Timedelta('-27 days +00:00:00')

In [13]:
X_1_train = X[(X['date'] < first_val_day) & (X['date'] >= first_train_day) ]
y_1_train = X_1_train[['demand']]
X_1_val = X[(X['date'] >= first_val_day) & (X['date'] < first_test_day)]
y_1_val = X_1_val[['demand']]
X_1_test = X[X['date'] >= first_test_day]

In [14]:
print(pd.Timestamp(X_1_test['date'].values.max()) )
print(pd.Timestamp(X_1_test['date'].values.min()))

2016-05-22 00:00:00
2016-04-25 00:00:00


In [15]:
X_1_test[['d', 'days_from_start']]

,d,days_from_start
0,d_1914,1913
1,d_1914,1913
2,d_1914,1913
3,d_1914,1913
4,d_1914,1913
...,...,...
853715,d_1941,1940
853716,d_1941,1940
853717,d_1941,1940
853718,d_1941,1940


In [16]:
print(X_1_train.shape)
print(y_1_train.shape)
print(X_1_val.shape)
print(y_1_val.shape)
print(X_1_test.shape)

(37915775, 73)
(37915775, 1)
(853632, 73)
(853632, 1)
(853720, 73)


In [17]:
del X

In [18]:
X_1_val.to_pickle("../01_preprocessed_data/X_1_val.pkl")
y_1_val.to_pickle("../01_preprocessed_data/y_1_val.pkl")
del X_1_val, y_1_val

In [19]:
X_1_test.to_pickle("../01_preprocessed_data/X_1_test.pkl")
del X_1_test

In [20]:
X_1_train.to_pickle("../01_preprocessed_data/X_1_train.pkl")
y_1_train.to_pickle("../01_preprocessed_data/y_1_train.pkl")
del X_1_train, y_1_train